
Assuma que se deseja prever a soma do número de diárias de carros a serem vendidas nos próximos 7 dias em cada agência, em função do preço médio a ser arbitrado para cada tipo de carro em cada agência.

Os dados históricos de vendas estão em um arquivo CSV (ou em uma tabela SQL) com as seguintes colunas e respectivos dados em cada linha, onde cada linha corresponde a uma venda:

-----

COD_AG: código da agência

NUM_NOTA: número da nota fiscal

DATA: data correspondente à venda

COD_TIPO_CARRO: código do tipo de carro que foi alugado

NUM_DIARIAS: número de diárias vendidas

VALOR: valor total da nota fiscal

Note que:

 * Múltiplas vendas do mesmo tipo de carro podem ocorrer no mesmo dia e na mesma agência, cada uma com preço potencialmente diferente.

 * O preço unitário de uma diária em uma venda pode ser calculado pela divisão VALOR/NUM_DIARIAS.

 * A DATA pode ser assumida como um número inteiro entre 1 e 7000, onde esse número corresponde ao número de dias desde o início da operação da empresa.

 * Você pode assumir que há aluguel de todos os tipos de carros em todas as agências ao longo de todos os dias do histórico.

Exercícios:

Escreva o código (usando SQL; Python com Pandas ou Spark; ou R) correspondente à preparação do seguinte dataframe (ou tabela SQL) para a criação de um modelo preditivo:

 * DATA_REF: data de referência (i.e. valores correspondem a 7 dias a partir dessa data)

 * COD_AG: código da agência

 * COD_TIPO_CARRO: código do tipo de carro

 * PRECO_MEDIO: preço médio ponderado por número de diárias do tipo de carro no período de referência e na agência

 * DELTA_PRECO_MEDIO: variação percentual do preço médio ponderado entre semana anterior (entre DATA_REF-7dias e DATA_REF-1 dia, incluindo os limites) e semana atual (entre DATA_REF e DATA_REF+6 dias, incluindo os limites)

 * NUM_DIARIAS_SEMANA_ANTERIOR: número total de diárias vendidas na semana anterior (entre DATA_REF-7dias e DATA_REF-1 dia, incluindo os limites)

 * NUM_DIARIAS_SEMANA: número total de diárias vendidas na semana atual (entre DATA_REF e DATA_REF+6 dias, incluindo os limites)


----

Proponha conjunto de “features” adicionais que possam ajudar na previsão futura, a serem adicionadas no dataframe. Note que há uma infinidade de “features” que podem ser úteis ao se agregar as vendas por semana e ao se considerar o histórico.

Explique como faria o treinamento e o teste do modelo.



In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [39]:
df_data = {
    'COD_AG': [1, 1, 2, 2, 1, 2, 1, 2],
    'NUM_NOTA': [1, 2, 3, 4, 5, 6, 7, 8],
    'DATA': [1, 2, 8, 9, 15, 16, 22, 23],
    'COD_TIPO_CARRO': [101, 102, 101, 103, 102, 104, 101, 103],
    'NUM_DIARIAS': [2, 3, 1, 2, 3, 1, 2, 2],
    'VALOR': [100, 150, 80, 120, 200, 90, 160, 140]
}



In [40]:
df = pd.DataFrame(df_data)
print(df)


   COD_AG  NUM_NOTA  DATA  COD_TIPO_CARRO  NUM_DIARIAS  VALOR
0       1         1     1             101            2    100
1       1         2     2             102            3    150
2       2         3     8             101            1     80
3       2         4     9             103            2    120
4       1         5    15             102            3    200
5       2         6    16             104            1     90
6       1         7    22             101            2    160
7       2         8    23             103            2    140


In [41]:
# Calculando preço médio por tipo de carro e agência
df['PRECO_UNITARIO'] = df['VALOR'] / df['NUM_DIARIAS']
preco_medio = df.groupby(['COD_AG', 'COD_TIPO_CARRO']).agg({'PRECO_UNITARIO': 'mean'}).reset_index()
preco_medio.rename(columns={'PRECO_UNITARIO': 'PRECO_MEDIO'}, inplace=True)

In [42]:
# Calculando a data de referência e semana anterior
df['DATA_REF'] = df['DATA'] + 7
df['DATA_SEMANA_ANTERIOR'] = df['DATA'] - 7

In [43]:
# Merge para adicionar o preço médio ao dataframe
df = pd.merge(df, preco_medio, on=['COD_AG', 'COD_TIPO_CARRO'], how='left')

In [44]:
# Calculando as métricas necessárias para cada linha
# Variação percentual do preço médio
df['DELTA_PRECO_MEDIO'] = (df['PRECO_MEDIO'] - df.groupby(['COD_AG', 'COD_TIPO_CARRO'])['PRECO_MEDIO'].shift(7)) / df.groupby(['COD_AG', 'COD_TIPO_CARRO'])['PRECO_MEDIO'].shift(7)



In [45]:
# Número total de diárias vendidas na semana anterior e na semana atual
diarias_semana_anterior = df.groupby(['COD_AG', 'COD_TIPO_CARRO', 'DATA_SEMANA_ANTERIOR']).agg({'NUM_DIARIAS': 'sum'}).reset_index()
diarias_semana_anterior.rename(columns={'NUM_DIARIAS': 'NUM_DIARIAS_SEMANA_ANTERIOR'}, inplace=True)



In [46]:


# Merge para adicionar o número de diárias vendidas na semana anterior e atual ao dataframe
df = pd.merge(df, diarias_semana_anterior, on=['COD_AG', 'COD_TIPO_CARRO', 'DATA_SEMANA_ANTERIOR'], how='left')
df = pd.merge(df, diarias_semana_atual, on=['COD_AG', 'COD_TIPO_CARRO', 'DATA_REF'], how='left')

# Exibindo o dataframe final
print(df)

   COD_AG  NUM_NOTA  DATA  COD_TIPO_CARRO  NUM_DIARIAS  VALOR  PRECO_UNITARIO  \
0       1         1     1             101            2    100       50.000000   
1       1         2     2             102            3    150       50.000000   
2       2         3     8             101            1     80       80.000000   
3       2         4     9             103            2    120       60.000000   
4       1         5    15             102            3    200       66.666667   
5       2         6    16             104            1     90       90.000000   
6       1         7    22             101            2    160       80.000000   
7       2         8    23             103            2    140       70.000000   

   DATA_REF  DATA_SEMANA_ANTERIOR  PRECO_MEDIO  DELTA_PRECO_MEDIO  \
0         8                    -6    65.000000                NaN   
1         9                    -5    58.333333                NaN   
2        15                     1    80.000000                N